In [1]:
import torch
import cv2 as cv
import os
import pandas as pd
import shutil
from PIL import Image

In [2]:

def model_eg():
    # Load a YOLOv5 model (options: yolov5n, yolov5s, yolov5m, yolov5l, yolov5x)
    model = torch.hub.load("ultralytics/yolov5", "yolov5n")  # Default: yolov5s

    # Define the input image source (URL, local file, PIL image, OpenCV frame, numpy array, or list)
    img = cv.imread("test_images/clear-gw.jpg", cv.IMREAD_COLOR)  # Example image
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

    # Perform inference (handles batching, resizing, normalization automatically)
    results = model(img)

    # Process the results (options: .print(), .show(), .save(), .crop(), .pandas())
    results.print()  # Print results to console
    # results.show()  # Display results in a window
    results.save()  # Save results to runs/detect/exp

## Getting the dataset ready

In [3]:
GTSRB_path = "./GTSRB/"

train_img_path = "./dataset/images/train"
val_img_path = "dataset/images/val"
train_lbl_path = "./dataset/labels/train"
val_lbl_path = "dataset/labels/val"

In [ ]:
def delete_dataset():
    folders = [train_img_path, val_img_path, train_lbl_path, val_lbl_path]
    for folder in folders:
        print("deleting from:", folder)
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)

# delete_dataset()

deleting from: ./dataset/images/train
deleting from: dataset/images/val
deleting from: ./dataset/labels/train
deleting from: dataset/labels/val


In [ ]:

train_val_split = 0.8


def convert_GTSRB_to_YOLO(dir, class_id):
    path = os.path.join(GTSRB_path, dir)
    csv_file_name = [file for file in os.listdir(path) if file.endswith(".csv")][0]
    csv_file_name = os.path.join(path, csv_file_name)

    df = pd.read_csv(csv_file_name, sep=";")
    # shuffle df
    df = df.sample(frac=1).reset_index(drop=True)

    # Filename    Width       Height     Roi.X1    Roi.Y1   Roi.X2  Roi.Y2 ClassId
    for index, row in df.iterrows():
        # copy img
        img_path = train_img_path
        lbl_path = train_lbl_path
        if index / len(df) > train_val_split:
            img_path = val_img_path
            lbl_path = val_lbl_path

        src_img_filename = row["Filename"]
        dst_img_filename = str(class_id) + "_" + src_img_filename
        dst_img_filename = dst_img_filename.replace('.ppm', '.png')

        img_src_path = os.path.join(path, src_img_filename)
        img_dst_path = os.path.join(img_path, dst_img_filename)

        img = Image.open(img_src_path)
        img.save(img_dst_path, "PNG")

        img_width = row["Width"]
        img_height = row["Height"]
        x1 = row["Roi.X1"]
        y1 = row["Roi.Y1"]
        x2 = row["Roi.X2"]
        y2 = row["Roi.Y2"]

        # convert to yolo
        x_center = ((x1 + x2) / 2) / img_width
        y_center = ((y1 + y2) / 2) / img_height
        bbox_width = (x2 - x1) / img_width
        bbox_height = (y2 - y1) / img_height

        label_filename = dst_img_filename.replace('.png', '.txt')
        label_path = os.path.join(lbl_path, label_filename)
        with open(label_path, 'w') as f:
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

# convert_GTSRB_to_YOLO("00013-give-way", 0)
# convert_GTSRB_to_YOLO("00014-stop", 1)
# convert_GTSRB_to_YOLO("00017-no-entry", 2)
# convert_GTSRB_to_YOLO("00018-warning", 3)